In [10]:
from roboflow import Roboflow

rf = Roboflow(api_key="jGXcSHCLXxRCAkyWA8Gj")
project = rf.workspace("sesac").project("monte_dataset")
version = project.version(1)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.0.238, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to monte_dataset-1 in yolov8:: 100%|██████████| 596/596 [00:00<00:00, 9646.65it/s]


In [18]:
import torch
from ultralytics import YOLO
import wandb
from wandb.integration.ultralytics import add_wandb_callback

# MPS 디바이스 사용 가능 여부 확인
print(f"MPS 사용 가능: {torch.backends.mps.is_available()}")
print(f"MPS 빌드: {torch.backends.mps.is_built()}")

# 디바이스 설정
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"사용 중인 디바이스: {device}")

# wandb 초기화
wandb.init(project="yolov8-character-detection", name="character_model_run")

# 사전 훈련된 모델 로드
model = YOLO("yolov8n.pt")  # 또는 "yolov8n.yaml"을 사용할 수 있습니다.

# wandb 콜백 추가
add_wandb_callback(model)

# 모델 학습
results = model.train(
    data="./monte_dataset-1/data.yaml",
    epochs=100,
    imgsz=640,
    batch=16,
    name="character_model",
    device=device,
    save_period=10,  # 10 에폭마다 모델 저장
    plots=True,  # 훈련 중 plot 생성
)

# 학습된 모델 저장
model.save("character_model.pt")

# wandb 실행 종료
model.export(format="onnx")

MPS 사용 가능: True
MPS 빌드: True
사용 중인 디바이스: mps


KeyboardInterrupt: 

In [19]:
model.export(format="onnx")

Ultralytics YOLOv8.0.238 🚀 Python-3.11.9 torch-2.4.1 MPS (Apple M3 Pro)

PyTorch: starting from '/opt/homebrew/runs/detect/character_model10/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 6, 8400) (6.0 MB)
requirements: Ultralytics requirement ['onnx>=1.12.0'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 11.8 MB/s eta 0:00:00 MB/s eta 0:00:01:01

requirements: AutoUpdate success ✅ 3.6s, installed 1 package: ['onnx>=1.12.0']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.16.2 opset 19...
ONNX: export success ✅ 74.6s, saved as '/opt/homebrew/runs/detect/character_model10/weights/best.onnx' (11.7 MB)

Export complete (94.8s)
Results saved to /opt/homebrew/runs/detect/character_model10/weights
Predict:         yolo predict task=detect model=/opt/homebrew/runs/detect/character_model10/weights/best.onnx imgsz=640  
Validate:        yolo val task=d

'/opt/homebrew/runs/detect/character_model10/weights/best.onnx'

In [16]:
import torch
from ultralytics import YOLO
import wandb
from wandb.integration.ultralytics import add_wandb_callback

# MPS 디바이스 사용 가능 여부 확인
print(f"MPS 사용 가능: {torch.backends.mps.is_available()}")
print(f"MPS 빌드: {torch.backends.mps.is_built()}")

# 디바이스 설정
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"사용 중인 디바이스: {device}")

# wandb 초기화 (이전 실행을 계속하려면 동일한 run_id를 사용)
wandb.init(
    project="yolov8-character-detection",
    name="character_model_run_resumed",
    resume="must",
)

# 이전 학습의 체크포인트에서 모델 로드
model = YOLO(
    "runs/detect/train/weights/last.pt"
)  # 경로를 실제 last.pt 파일 위치로 변경하세요

# wandb 콜백 추가
add_wandb_callback(model)

# 학습 재개
results = model.train(
    data="./monte_dataset-1/data.yaml",
    epochs=100,
    imgsz=640,
    batch=16,
    name="character_model_resumed",
    device=device,
    save_period=10,
    plots=True,
    resume="allow",
)

# 학습된 모델 저장
model.save("character_model_resumed.pt")

# wandb 실행 종료
wandb.finish()

MPS 사용 가능: True
MPS 빌드: True
사용 중인 디바이스: mps


KeyboardInterrupt: 

In [13]:
# 검증 데이터셋에 대한 모델 성능 평가
results = model.val()

print(results.box.map)  # mAP@0.5:0.95
print(results.box.map50)  # mAP@0.5
print(results.box.map75)  # mAP@0.75

Ultralytics YOLOv8.0.238 🚀 Python-3.11.9 torch-2.4.1 MPS (Apple M3 Pro)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /Users/syshin/Desktop/Syshin/Study/MLDL/help/monte_yolo/monte_dataset-1/valid/labels.cache... 25 images, 0 backgrounds, 0 corrupt: 100%|██████████| 25/25 [00:00<?, ?it/s]


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/2 [00:00<?, ?it/s]

WARNING ⚠️ NMS time limit 1.300s exceeded


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  50%|█████     | 1/2 [04:32<04:32, 272.17s/it]

WARNING ⚠️ NMS time limit 0.950s exceeded


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [07:38<00:00, 229.19s/it]


                   all         25        127       0.31      0.186      0.316       0.15
       character_brown         25        123      0.536      0.122      0.356      0.238
                 medal         25          4     0.0833       0.25      0.277     0.0616


Generating Visualizations for batch-1/1: 100%|██████████| 16/16 [00:01<00:00, 11.61it/s]


AssertionError: We don't support referring to media files across runs.

In [14]:
results

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0xfa981e690>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [22]:
from ultralytics import YOLO
import cv2
import numpy as np

# 학습된 모델 로드
model = YOLO("/opt/homebrew/runs/detect/character_model10/weights/best.pt")

# 비디오 파일 열기 (0은 웹캠을 의미합니다)
video_path = "data/sample2.mp4"
cap = cv2.VideoCapture(video_path)

# 비디오 저장을 위한 설정
output_path = "output_video.mp4"
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
codec = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_path, codec, fps, (width, height))

# 비디오 프레임 처리
while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    # YOLOv8 모델로 객체 감지
    results = model(frame)

    # 결과 시각화
    annotated_frame = results[0].plot()

    # 프레임 저장
    out.write(annotated_frame)

    # 결과 표시 (실시간 모니터링을 원하는 경우)
    cv2.imshow("YOLOv8 Inference", annotated_frame)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# 리소스 해제
cap.release()
out.release()
cv2.destroyAllWindows()


0: 512x640 5 character_browns, 68.3ms
Speed: 6.0ms preprocess, 68.3ms inference, 9.4ms postprocess per image at shape (1, 3, 512, 640)

0: 512x640 5 character_browns, 65.3ms
Speed: 1.5ms preprocess, 65.3ms inference, 0.3ms postprocess per image at shape (1, 3, 512, 640)

0: 512x640 5 character_browns, 40.3ms
Speed: 1.2ms preprocess, 40.3ms inference, 0.3ms postprocess per image at shape (1, 3, 512, 640)

0: 512x640 5 character_browns, 33.3ms
Speed: 1.5ms preprocess, 33.3ms inference, 0.3ms postprocess per image at shape (1, 3, 512, 640)

0: 512x640 5 character_browns, 35.0ms
Speed: 2.4ms preprocess, 35.0ms inference, 0.3ms postprocess per image at shape (1, 3, 512, 640)

0: 512x640 5 character_browns, 33.3ms
Speed: 1.4ms preprocess, 33.3ms inference, 0.3ms postprocess per image at shape (1, 3, 512, 640)

0: 512x640 5 character_browns, 170.0ms
Speed: 12.2ms preprocess, 170.0ms inference, 0.6ms postprocess per image at shape (1, 3, 512, 640)

0: 512x640 5 character_browns, 36.1ms
Speed:

In [27]:
from ultralytics import YOLO
import cv2
import numpy as np

# 학습된 모델 로드
model = YOLO("/opt/homebrew/runs/detect/character_model10/weights/best.pt")

# 비디오 파일 열기
video_path = "data/youtube_sample.mp4"
cap = cv2.VideoCapture(video_path)

# 비디오 저장을 위한 설정
output_path = "tests/youtube_sample.mp4"
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
codec = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_path, codec, fps, (width, height))

# 트래킹을 위한 설정
track_history = {}  # 각 객체의 트래킹 히스토리를 저장할 딕셔너리
id_mapping = {}  # 원본 ID를 새로운 ID로 매핑
next_id = 1  # 다음에 할당할 ID

# 비디오 프레임 처리
frame_count = 0
while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    frame_count += 1

    # YOLOv8 모델로 객체 감지 및 트래킹
    results = model.track(frame, persist=True)

    # 결과 시각화
    annotated_frame = results[0].plot()

    # 트래킹 결과 처리 및 시각화
    if results[0].boxes.id is not None:
        boxes = results[0].boxes.xywh.cpu()
        track_ids = results[0].boxes.id.int().cpu().tolist()

        for box, track_id in zip(boxes, track_ids):
            if track_id not in id_mapping:
                id_mapping[track_id] = next_id
                next_id += 1

            new_id = id_mapping[track_id]
            x, y, w, h = box

            if new_id not in track_history:
                track_history[new_id] = []

            track_history[new_id].append((float(x), float(y), frame_count))

            # 오래된 포인트 제거 (5초 이상 지난 포인트)
            track_history[new_id] = [
                p for p in track_history[new_id] if frame_count - p[2] < 5 * fps
            ]

            # 트래킹 경로 그리기
            if len(track_history[new_id]) > 1:
                points = np.array(
                    [(int(p[0]), int(p[1])) for p in track_history[new_id]],
                    dtype=np.int32,
                )
                cv2.polylines(
                    annotated_frame,
                    [points],
                    isClosed=False,
                    color=(0, 255, 0),
                    thickness=2,
                )

            # ID 표시
            cv2.putText(
                annotated_frame,
                f"ID: {new_id}",
                (int(x), int(y) - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.9,
                (0, 255, 0),
                2,
            )

    # 프레임 저장
    out.write(annotated_frame)

    # 결과 표시 (실시간 모니터링을 원하는 경우)
    cv2.imshow("YOLOv8 Tracking", annotated_frame)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# 리소스 해제
cap.release()
out.release()
cv2.destroyAllWindows()


0: 480x640 1 character_brown, 1 medal, 72.5ms
Speed: 9.3ms preprocess, 72.5ms inference, 10.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 character_brown, 1 medal, 34.5ms
Speed: 1.4ms preprocess, 34.5ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 28.7ms
Speed: 1.4ms preprocess, 28.7ms inference, 0.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 23.3ms
Speed: 1.3ms preprocess, 23.3ms inference, 0.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 23.9ms
Speed: 1.4ms preprocess, 23.9ms inference, 0.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 26.4ms
Speed: 1.3ms preprocess, 26.4ms inference, 0.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 22.9ms
Speed: 1.5ms preprocess, 22.9ms inference, 0.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 character_brown, 19.6ms
Speed: 1

In [26]:
from collections import defaultdict
import cv2
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator, colors

# 트래킹 히스토리 초기화
track_history = defaultdict(lambda: [])

# 학습된 모델 로드
model = YOLO("/opt/homebrew/runs/detect/character_model10/weights/best.pt")

# 비디오 파일 열기
video_path = "data/sample2.mp4"
cap = cv2.VideoCapture(video_path)

# 비디오 속성 가져오기
w, h, fps = (
    int(cap.get(x))
    for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS)
)

# 출력 비디오 설정
out = cv2.VideoWriter(
    "character-tracking-output.avi", cv2.VideoWriter_fourcc(*"MJPG"), fps, (w, h)
)

while True:
    ret, im0 = cap.read()
    if not ret:
        print("비디오 처리가 완료되었거나 프레임을 읽을 수 없습니다.")
        break

    # Annotator 객체 생성
    annotator = Annotator(im0, line_width=2)

    # 객체 감지 및 추적 수행
    results = model.track(im0, persist=True)

    if results[0].boxes.id is not None:
        boxes = results[0].boxes.xywh.cpu()
        track_ids = results[0].boxes.id.int().cpu().tolist()

        # 각 감지된 객체에 대해
        for box, track_id in zip(boxes, track_ids):
            x, y, w, h = box
            color = colors(int(track_id), True)

            # 바운딩 박스 그리기
            annotator.box_label(
                [x - w / 2, y - h / 2, x + w / 2, y + h / 2],
                f"ID: {track_id}",
                color=color,
            )

            # 트래킹 히스토리 업데이트 및 그리기
            track = track_history[track_id]
            track.append((float(x), float(y)))  # 중심점 저장
            if len(track) > 30:  # 최대 30개의 포인트만 유지
                track.pop(0)

            # 트래킹 경로 그리기
            points = [(int(x), int(y)) for x, y in track]
            cv2.polylines(
                im0,
                [np.array(points, dtype=np.int32)],
                isClosed=False,
                color=color,
                thickness=2,
            )

    # 프레임 저장 및 표시
    out.write(im0)
    cv2.imshow("Character Tracking", im0)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# 리소스 해제
out.release()
cap.release()
cv2.destroyAllWindows()


0: 512x640 5 character_browns, 36.2ms
Speed: 2.1ms preprocess, 36.2ms inference, 0.6ms postprocess per image at shape (1, 3, 512, 640)

0: 512x640 5 character_browns, 46.9ms
Speed: 1.2ms preprocess, 46.9ms inference, 0.3ms postprocess per image at shape (1, 3, 512, 640)

0: 512x640 5 character_browns, 32.0ms
Speed: 1.2ms preprocess, 32.0ms inference, 0.4ms postprocess per image at shape (1, 3, 512, 640)

0: 512x640 5 character_browns, 58.1ms
Speed: 4.1ms preprocess, 58.1ms inference, 0.3ms postprocess per image at shape (1, 3, 512, 640)

0: 512x640 5 character_browns, 50.0ms
Speed: 1.3ms preprocess, 50.0ms inference, 0.3ms postprocess per image at shape (1, 3, 512, 640)

0: 512x640 5 character_browns, 32.0ms
Speed: 1.1ms preprocess, 32.0ms inference, 0.4ms postprocess per image at shape (1, 3, 512, 640)

0: 512x640 5 character_browns, 37.1ms
Speed: 1.4ms preprocess, 37.1ms inference, 0.3ms postprocess per image at shape (1, 3, 512, 640)

0: 512x640 5 character_browns, 34.0ms
Speed: 1.